# 1D CNN, Bi-lstm(keras embedding) - k-fold 추가(label 0,1/불균형 맞추기)

In [ ]:
import pandas as pd

dff=pd.read_excel("/content/spell_labeling.xlsx")
dff

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,label,sentence
0,0,0,0,0,창문 구조 자체가 방음이 잘되지 않고 찬바람이 들어오는 것 같아요.
1,1,1,1,0,지금 새벽 네시가 다 되어 가는데 밖에 경찰차 소리가 엄청나게 크게 들리네요?.
2,2,2,2,0,난방도 기능은 존재하는데 되는지를 모르겠네요.
3,3,3,3,0,별로 춥지 않아서 컴플레인은 걸지 않았지만 겨울철 되면 문제가 커질 것 같고요.
4,4,4,4,0,그리고 화장실 문이 심하게 덜 렁 꺼리고 콘센트가 거의 없어서 매우 불편했습니다.
...,...,...,...,...,...
41350,41350,41350,41350,1,젛았습니다습니다
41351,41351,41351,41351,1,즇앗씁니다!니다!
41352,41352,41352,41352,1,좋ㅉㅗㅎ 앗습니다
41353,41353,41353,41353,1,펺습니다습니다


In [ ]:
feature=dff['sentence']

In [ ]:
dff.loc[11224,'label']=1
dff.loc[12739,'label']=1

label 2값 제거

In [ ]:
df=dff[dff.label != 2]
df=df.reset_index()
df

,index,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,label,sentence
0,0,0,0,0,0,창문 구조 자체가 방음이 잘되지 않고 찬바람이 들어오는 것 같아요.
1,1,1,1,1,0,지금 새벽 네시가 다 되어 가는데 밖에 경찰차 소리가 엄청나게 크게 들리네요?.
2,2,2,2,2,0,난방도 기능은 존재하는데 되는지를 모르겠네요.
3,3,3,3,3,0,별로 춥지 않아서 컴플레인은 걸지 않았지만 겨울철 되면 문제가 커질 것 같고요.
4,4,4,4,4,0,그리고 화장실 문이 심하게 덜 렁 꺼리고 콘센트가 거의 없어서 매우 불편했습니다.
...,...,...,...,...,...,...
39909,41350,41350,41350,41350,1,젛았습니다습니다
39910,41351,41351,41351,41351,1,즇앗씁니다!니다!
39911,41352,41352,41352,41352,1,좋ㅉㅗㅎ 앗습니다
39912,41353,41353,41353,41353,1,펺습니다습니다


In [ ]:
# from imblearn.under_sampling import *
feature=df['sentence']
# X_samp, y_samp = OneSidedSelection(random_state=0).fit_sample(df0, df1)


In [ ]:
r = open('/content/한국어불용어100.txt', mode='rt', encoding='utf-8')
stopwords=[]
z=r.readlines()
for i in range(len(z)):
  stopwords.append(z[i].split('\t')[0])

In [ ]:
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

In [ ]:
import re
feature = feature.str.replace('<br />',' ')

feature=feature.apply(lambda x : re.sub('[^a-zA-Zㄱ-ㅎ|ㅏ-ㅣ|가-힣]',' ',str(x)))

In [ ]:
import numpy as np
feature = feature.apply(remove_stopwords)
feature = np.array([x for x in feature])

In [ ]:
from keras.utils import np_utils
from tensorflow.keras.utils import plot_model, to_categorical

y_train=df['label'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature,
                                                   y_train,
                                                   test_size=0.2,
                                                   random_state=42)

In [ ]:
from keras_preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer

embedding_dim=300
max_length=72
padding_type='post'

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train) 
word_index = tokenizer.word_index

vocab_size=len(word_index)+1 #vocabulary 크기 설정
print(vocab_size)

23369


In [ ]:
from keras_preprocessing.sequence import pad_sequences 

#데이터를 sequence로 변환하고 padding 해주기
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length) 

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

print(train_padded.shape, test_padded.shape)

(31931, 72) (7983, 72)


In [ ]:
#불균형 맞춰주기
from imblearn.under_sampling import *
from imblearn.over_sampling import *


train_padded, y_train = OneSidedSelection(random_state=0).fit_sample(train_padded, y_train) 
#train_padded, y_train = SMOTEENN(random_state=0).fit_sample(train_padded, y_train) 0.794 #오버샘플링(가중치를 증가)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional, GlobalMaxPooling1D, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam

epoch_val = 10
model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        #첫번째 모델
        # Bidirectional(LSTM(64, return_sequences=True)),
        # Bidirectional(LSTM(64, return_sequences = True)),
        # Bidirectional(LSTM(64)),
        # Dense(2, activation='sigmoid')
        #0.9296
        
        #두번째 모델(1D CNN)
        Conv1D(32, 7, activation='relu'),
        MaxPooling1D(5),
        Conv1D(32, 7, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(2,activation='sigmoid')
        #0.9359889765752224

        #3번째 모델
        # Bidirectional(LSTM(128, return_sequences=True)),
        # LSTM(128),
        # Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'),
        # GlobalMaxPooling1D(),
        # Dropout(.5),
        # Dense(128,activation='swish'),
        # # Dropout(.5),
        # Dense(2,activation='sigmoid')
    ])

    
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
# 계층 교차 검증
n_fold = 5  
seed = 42
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical

cv = StratifiedKFold(n_splits = n_fold, shuffle=True, random_state=seed)

# 테스트데이터의 예측값 담을 곳 생성
test_y = np.zeros((test_padded.shape[0], 2))

# 조기 종료 옵션 추가
es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                   verbose=1, mode='min', baseline=None, restore_best_weights=True)

for i, (i_trn, i_val) in enumerate(cv.split(train_padded, y_train), 1):
    print(f'training model for CV #{i}')

    model.fit(train_padded[i_trn], 
            to_categorical(y_train[i_trn]),
            validation_data=(train_padded[i_val], to_categorical(y_train[i_val])),
            epochs=10,
            batch_size=256,
            callbacks=[es])     # 조기 종료 옵션
                      
    test_y += model.predict(test_padded) / n_fold    # 나온 예측값들을 교차 검증 횟수로 나눈다

training model for CV #1
Epoch 1/10
95/95 [==============================] - 41s 115ms/step - loss: 0.4913 - accuracy: 0.7982 - val_loss: 0.2133 - val_accuracy: 0.9040
Epoch 2/10
95/95 [==============================] - 10s 109ms/step - loss: 0.1411 - accuracy: 0.9468 - val_loss: 0.1749 - val_accuracy: 0.9305
Epoch 3/10
95/95 [==============================] - 10s 108ms/step - loss: 0.0527 - accuracy: 0.9832 - val_loss: 0.1977 - val_accuracy: 0.9304
Epoch 4/10
95/95 [==============================] - 10s 109ms/step - loss: 0.0252 - accuracy: 0.9938 - val_loss: 0.2215 - val_accuracy: 0.9301
Epoch 5/10
95/95 [==============================] - 10s 107ms/step - loss: 0.0159 - accuracy: 0.9961 - val_loss: 0.2510 - val_accuracy: 0.9254
Restoring model weights from the end of the best epoch.
Epoch 00005: early stopping
training model for CV #2
Epoch 1/10
95/95 [==============================] - 10s 109ms/step - loss: 0.0922 - accuracy: 0.9676 - val_loss: 0.0617 - val_accuracy: 0.9793
Epoch 2/

In [ ]:
y_pred_proba1=[]
for i in range(len(test_y)):
  if test_y[i][0]>test_y[i][1]:
    y_pred_proba1.append(0)
  elif test_y[i][1]>test_y[i][0]:
    y_pred_proba1.append(1)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

acc = accuracy_score(y_test, y_pred_proba1)

In [ ]:
print(acc)

0.9316046599022924


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(confusion_matrix(y_test, y_pred_proba1))
print(classification_report(y_test, y_pred_proba1, target_names=['0', '1']))

[[1172  301]
 [ 245 6265]]
              precision    recall  f1-score   support

           0       0.83      0.80      0.81      1473
           1       0.95      0.96      0.96      6510

    accuracy                           0.93      7983
   macro avg       0.89      0.88      0.88      7983
weighted avg       0.93      0.93      0.93      7983



In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

print(roc_auc_score(y_test, y_pred_proba1))

0.8790103584959378
